In [ ]:
!pip install -U adapter-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install torch torchvision torchaudio -f https://download.pytorch.org/whl/cu111/torch_stable.html

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.pytorch.org/whl/cu111/torch_stable.html


In [ ]:
from transformers import AutoConfig, AutoModelWithHeads
import torch
from transformers import RobertaTokenizer
import transformers.adapters.composition as ac
import pandas as pd
from tqdm import tqdm
from scipy.stats import pearsonr
import random

In [ ]:
val_data = pd.read_csv('/content/drive/MyDrive/NLP_final_pro/data/messages_dev_features_ready_for_WS_2022.tsv', sep='\t')

In [ ]:
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

config = AutoConfig.from_pretrained(
    "roberta-base",
    num_labels=1
)

model = AutoModelWithHeads.from_pretrained(
    "roberta-base",
    config=config,
)

/usr/local/lib/python3.10/dist-packages/transformers/adapters/models/roberta/adapter_model.py:274: FutureWarning: This class has been renamed to `RobertaAdapterModel` in v3. Please use the new class instead as this class might be removed in a future version.
  warnings.warn(


/usr/local/lib/python3.10/dist-packages/transformers/adapters/models/roberta/adapter_model.py:252: FutureWarning: This class has been renamed to `RobertaAdapterModel` in v3. Please use the new class instead as this class might be removed in a future version.
  warnings.warn(
Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModelWithHeads: ['lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaModelWithHeads from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModelWithHeads from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification mo

In [ ]:
""" load adapters """
empathy_adapter_path = "/content/drive/MyDrive/NLP_final_pro/trained_adapters/EMP_emotion_stack"
distress_adapter_path = "/content/drive/MyDrive/NLP_final_pro/trained_adapters/DIS_emotion_stack"

empathy_adapter = model.load_adapter(empathy_adapter_path, load_as=empathy_adapter_path.split('/')[-1])
distress_adapter = model.load_adapter(distress_adapter_path, load_as=distress_adapter_path.split('/')[-1])

In [ ]:
model.set_active_adapters(ac.Parallel(empathy_adapter, distress_adapter))
model.active_head = ['EMP_emotion_stack', 'DIS_emotion_stack']

In [ ]:
""" load pretrained adapter composition """

from transformers.adapters.composition import Fuse


fusion_path = f"/content/drive/MyDrive/NLP_final_pro/trained_adapters/EpitomeFusion-distress"

# load each individual adapter
dis_er_adapter = model.load_adapter(
                fusion_path + '/distress-emotional-reactions')
dis_ex_adapter = model.load_adapter(
                fusion_path + '/distress-explorations')
dis_ip_adapter = model.load_adapter(
                fusion_path + '/distress-interpretations')

# load adapter fusion
model.load_adapter_fusion(
    fusion_path
)

# set active adapters
model.set_active_adapters(Fuse(dis_er_adapter, dis_ex_adapter, dis_ip_adapter))

# # load head
# path, distress_adapter = model.load_head(fusion_path)

model.active_head = 'EpitomeFusion-distress'

In [ ]:
""" load pretrained adapter composition: empathy """

fusion_path = f"/content/drive/MyDrive/NLP_final_pro/trained_adapters/EpitomeFusion-empathy"

# load each individual adapter
emp_er_adapter = model.load_adapter(
                fusion_path + '/empathy-emotional-reactions'
            )
emp_ex_adapter = model.load_adapter(
                fusion_path + '/empathy-explorations'
            )
emp_ip_adapter = model.load_adapter(
                fusion_path + '/empathy-interpretations'
            )

# load adapter fusion
model.load_adapter_fusion(
    fusion_path
)

# set active adapters
model.set_active_adapters(Fuse(emp_er_adapter, emp_ex_adapter, emp_ip_adapter))

# # load head
# path, empathy_adapter = model.load_head(fusion_path)

In [ ]:
model.active_head = 'EpitomeFusion-empathy'
model.active_head

'EpitomeFusion-empathy'

In [ ]:
def predict_sen(sentence):
    tokens = tokenizer.tokenize(sentence)
    input_ids = torch.tensor(tokenizer.convert_tokens_to_ids(tokens)).to('cuda')

    # move the model to the GPU
    model.to('cuda')

    # set active adapters
    model.set_active_adapters(Fuse(dis_er_adapter, dis_ex_adapter, dis_ip_adapter))
    model.active_head = 'EpitomeFusion-distress'
    outputs = model(input_ids)
    dis_score_f = outputs.logits[0][0].tolist()

    # set active adapters
    model.set_active_adapters(Fuse(emp_er_adapter, emp_ex_adapter, emp_ip_adapter))
    model.active_head = 'EpitomeFusion-empathy'
    outputs = model(input_ids)
    emp_score_f = outputs.logits[0][0].tolist()

    # stack
    model.set_active_adapters(ac.Parallel(empathy_adapter, distress_adapter))
    model.active_head = ['EMP_emotion_stack', 'DIS_emotion_stack']
    outputs = model(input_ids)

    emp_score_s = outputs[0].logits[0][0].tolist()
    dis_score_s = outputs[1].logits[0][0].tolist()

    # move the model back to the CPU
    model.to('cpu')

    return emp_score_f, dis_score_f, emp_score_s, dis_score_s

In [ ]:
sen = val_data['essay'].tolist()
random_five = random.sample(sen, 5)

In [ ]:
for se in random_five:
  emp_score_f, dis_score_f, emp_score_s, dis_score_s = predict_sen(se)
  print(se)
  print("emp_score_f =",emp_score_f, "dis_score_f =",dis_score_f,"emp_score_s =", emp_score_s,"dis_score_s =" ,dis_score_s)

The band that was playing during the attack on the Bataclan in France returned there to honor the victims recently. It was very sad because it was such a large and brutal massacre. So many lives were lost that day and for no reason what so ever. It is just very depressing to think about what those people must have went through.
emp_score_f = 5.878396987915039 dis_score_f = 4.984640598297119 emp_score_s = 6.087838172912598 dis_score_s = 4.8839030265808105
It looks like some rappers are fighting over who got credit to some song that was written. It seems like some simple petty argument that needs to be decided in court. That being said, a million dollars is a lot of money. If someone is cheated out of this money, they deserve to get it back in my opnion.
emp_score_f = 3.265986680984497 dis_score_f = 2.274444341659546 emp_score_s = 2.9244091510772705 dis_score_s = 1.9206278324127197
I am moved to tears reading this. People are not having it easy in other parts of the world. It made me mor

In [ ]:
re_sen = ["Im sorry but I cant imagine what it must be like to lose a grandmother Sarah I hope you can find the strength to cope with your loss",
          "I dont really know much about your hometown Mark so I cant say I understand how hard it is to leave But Im here to listen if you need to talk",
          "Ive never worked in a busy city like New York Jennifer so I dont know what its like to adjust to a new job there But Im here to support you in any way I can",
          "Ive never experienced rush hour traffic in Los Angeles John so I cant fully appreciate how frustrating it must be But Im here to help if you need anything",
          "Ive never been in a situation where I lost my job Maria so I cant say I understand how you feel But please let me know if theres anything I can do to help"]

In [ ]:
for se in re_sen:
  emp_score_f, dis_score_f, emp_score_s, dis_score_s = predict_sen(se)
  print(se)
  print("emp_score_f =",emp_score_f, "dis_score_f =",dis_score_f,"emp_score_s =", emp_score_s,"dis_score_s =" ,dis_score_s)

Im sorry but I cant imagine what it must be like to lose a grandmother Sarah I hope you can find the strength to cope with your loss
emp_score_f = 5.4678192138671875 dis_score_f = 4.208892822265625 emp_score_s = 5.699822902679443 dis_score_s = 4.61144495010376
I dont really know much about your hometown Mark so I cant say I understand how hard it is to leave But Im here to listen if you need to talk
emp_score_f = 5.165220737457275 dis_score_f = 2.9229483604431152 emp_score_s = 5.0348029136657715 dis_score_s = 3.05773663520813
Ive never worked in a busy city like New York Jennifer so I dont know what its like to adjust to a new job there But Im here to support you in any way I can
emp_score_f = 4.1959147453308105 dis_score_f = 2.8213045597076416 emp_score_s = 3.4431838989257812 dis_score_s = 2.635380983352661
Ive never experienced rush hour traffic in Los Angeles John so I cant fully appreciate how frustrating it must be But Im here to help if you need anything
emp_score_f = 5.009432315

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
